In [1]:
from pathlib import Path
from tqdm import tqdm
from virtualizarr import open_virtual_dataset, open_virtual_mfdataset
from virtualizarr.parsers import HDFParser
from virtualizarr.registry import ObjectStoreRegistry
from icechunk.xarray import to_icechunk
from dask.diagnostics import ProgressBar

import obstore as obs
import xarray as xr
import icechunk
import hvplot.xarray
import xclim
import matplotlib.pyplot as plt
import pandas as pd
import pyproj

import warnings
warnings.filterwarnings(
  "ignore",
  message="Numcodecs codecs are not in the Zarr version 3 specification*",
  category=UserWarning
)
icechunk.set_logs_filter(
    "icechunk::storage::object_store=error"
)  # remove the warning due to the local icechunk store

In [2]:
ROOT_DIR = Path("/Volumes/T7")
RAW_FILES_DIR = ROOT_DIR / "raw_files/hackathon-meteo-france"

BUCKETS_DIR = ROOT_DIR / "buckets"
ICECHUNK_BUCKET = BUCKETS_DIR / "hackathon-meteo-france"
PREFIX_ANASTASIA = "REA/ANASTASIA"
PREFIX_RCM = "RCM"

PREFIX_PROCESSED_OBS = "processed/observations"

prefix = "file:///Volumes/"

In [3]:
locations = {
    "paris":   {"lat": 48.85341,  "lon": 2.3488},
    "toulouse": {"lat": 43.60426,  "lon": 1.44367},
    "marseille": {"lat": 43.29695, "lon": 5.38107},
    "lyon":    {"lat": 45.74846,  "lon": 4.84671},
    "brest":   {"lat": 48.3904,   "lon": -4.4861},
    "annecy":  {"lat": 45.8992,   "lon": 6.1294}
}

In [4]:
storage = icechunk.local_filesystem_storage(
    # optional: we can add a prefix here
    path=ICECHUNK_BUCKET / PREFIX_RCM
)
new_repo = icechunk.Repository.open(
    storage,
    authorize_virtual_chunk_access={prefix: None}
)
new_session = new_repo.readonly_session("main")

dt = xr.open_datatree(new_session.store, engine='zarr', zarr_format=3, chunks={})
dt

<xarray.DataTree>
Group: /
├── Group: /hussAdjust
│       Dimensions:                      (source_id: 17, time: 23376, y: 134, x: 143)
│       Coordinates:
│         * source_id                    (source_id) object 136B 'EC-Earth3:r1i1p1f1:...
│         * time                         (time) datetime64[ns] 187kB 1951-01-01T12:00...
│         * y                            (y) float64 1kB 1.617e+06 ... 2.681e+06
│         * x                            (x) float64 1kB 6e+04 6.8e+04 ... 1.196e+06
│           input_driving_source_id      (source_id) object 136B dask.array<chunksize=(17,), meta=np.ndarray>
│           input_source_id              (source_id) object 136B dask.array<chunksize=(17,), meta=np.ndarray>
│           lon                          (y, x) float64 153kB dask.array<chunksize=(134, 143), meta=np.ndarray>
│           input_driving_variant_label  (source_id) object 136B dask.array<chunksize=(17,), meta=np.ndarray>
│           lat                          (y, x) float64 153kB dask.array<chunksize=(134, 143), meta=np.ndarray>
│       Data variables:
│           LambertParisII               (source_id) int32 68B dask.array<chunksize=(1,), meta=np.ndarray>
│           hussAdjust                   (source_id, time, y, x) float32 30GB dask.array<chunksize=(1, 128, 134, 143), meta=np.ndarray>
│       Attributes: (12/51)
│           summary:                         Data were post-processed by the Météo-Fr...
│           Conventions:                     CF-1.11 ACDD-1.3
│           standard_name_vocabulary:        CF Standard Name Table v81
│           input_domain:                    Europe
│           input_domain_id:                 EUR-12
│           project:                         Socle-FR-Metro-Climat-2025
│           ...                              ...
│           time_coverage_duration:          P1Y
│           time_coverage_resolution:        P1D
│           product:                         bias-adjusted output
│           product_version:                 v1
│           comment:                         Preliminary data released in the Fall 20...
│           keywords:                        humidity, climate, France, near-surface ...
├── Group: /tasmaxAdjust
│       Dimensions:                      (source_id: 17, time: 23376, y: 134, x: 143)
│       Coordinates:
│         * source_id                    (source_id) object 136B 'EC-Earth3:r1i1p1f1:...
│         * time                         (time) datetime64[ns] 187kB 1951-01-01T12:00...
│         * y                            (y) float64 1kB 1.617e+06 ... 2.681e+06
│         * x                            (x) float64 1kB 6e+04 6.8e+04 ... 1.196e+06
│           lon                          (y, x) float64 153kB dask.array<chunksize=(134, 143), meta=np.ndarray>
│           input_source_id              (source_id) object 136B dask.array<chunksize=(17,), meta=np.ndarray>
│           input_driving_source_id      (source_id) object 136B dask.array<chunksize=(17,), meta=np.ndarray>
│           input_driving_variant_label  (source_id) object 136B dask.array<chunksize=(17,), meta=np.ndarray>
│           lat                          (y, x) float64 153kB dask.array<chunksize=(134, 143), meta=np.ndarray>
│       Data variables:
│           LambertParisII               (source_id) int32 68B dask.array<chunksize=(1,), meta=np.ndarray>
│           tasmaxAdjust                 (source_id, time, y, x) float32 30GB dask.array<chunksize=(1, 128, 134, 143), meta=np.ndarray>
│       Attributes: (12/50)
│           Conventions:                     CF-1.11 ACDD-1.3
│           standard_name_vocabulary:        CF Standard Name Table v81
│           input_domain:                    Europe
│           input_domain_id:                 EUR-12
│           project:                         Socle-FR-Metro-Climat-2025
│           grid_resolution:                 8-km
│           ...                              ...
│           time_coverage_duration:          P1Y
│           time_coverage_res

In [5]:
variables = list(dt.children)
variables

['hussAdjust',
 'tasmaxAdjust',
 'sfcWindAdjust',
 'prAdjust',
 'tasminAdjust',
 'tasAdjust']

In [6]:
def select_single_loc(ds, loc_name: str, loc: dict):
    proj = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:27572", always_xy=True)
    _x, _y = proj.transform(xx=loc["lon"], yy=loc["lat"])
    loc_lambert=dict(x=_x, y=_y)

    return ds.sel(**loc_lambert, method='nearest').expand_dims({"location": [loc_name]})

def select_timeseries(ds):
    timeseries = [
        select_single_loc(ds, loc_name=loc_name, loc=loc)
        for loc_name, loc in tqdm(locations.items())
    ]
    with xr.set_options(use_new_combine_kwarg_defaults=True):
        ds = xr.concat(timeseries, dim="location", coords="all")
    return ds

In [13]:
datasets = [
    select_timeseries(dt[var].dataset)
    for var in variables
]
ds = xr.merge(datasets, compat="override")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 96.34it/s]


In [14]:
ds

<xarray.Dataset> Size: 57MB
Dimensions:                      (location: 6, source_id: 17, time: 23376)
Coordinates:
  * location                     (location) object 48B 'paris' ... 'annecy'
  * source_id                    (source_id) object 136B 'EC-Earth3:r1i1p1f1:...
  * time                         (time) datetime64[ns] 187kB 1951-01-01T12:00...
    input_driving_source_id      (location, source_id) object 816B dask.array<chunksize=(1, 17), meta=np.ndarray>
    input_source_id              (location, source_id) object 816B dask.array<chunksize=(1, 17), meta=np.ndarray>
    lon                          (location) float64 48B dask.array<chunksize=(1,), meta=np.ndarray>
    x                            (location) float64 48B 6.04e+05 ... 8.92e+05
    input_driving_variant_label  (location, source_id) object 816B dask.array<chunksize=(1, 17), meta=np.ndarray>
    lat                          (location) float64 48B dask.array<chunksize=(1,), meta=np.ndarray>
    y                            (location) float64 48B 2.425e+06 ... 2.105e+06
Data variables:
    LambertParisII               (location, source_id) int32 408B dask.array<chunksize=(1, 1), meta=np.ndarray>
    hussAdjust                   (location, source_id, time) float32 10MB dask.array<chunksize=(1, 1, 128), meta=np.ndarray>
    tasmaxAdjust                 (location, source_id, time) float32 10MB dask.array<chunksize=(1, 1, 128), meta=np.ndarray>
    sfcWindAdjust                (location, source_id, time) float32 10MB dask.array<chunksize=(1, 1, 128), meta=np.ndarray>
    prAdjust                     (location, source_id, time) float32 10MB dask.array<chunksize=(1, 1, 128), meta=np.ndarray>
    tasminAdjust                 (location, source_id, time) float32 10MB dask.array<chunksize=(1, 1, 128), meta=np.ndarray>
    tasAdjust                    (location, source_id, time) float32 10MB dask.array<chunksize=(1, 1, 128), meta=np.ndarray>
Attributes: (12/51)
    summary:                         Data were post-processed by the Météo-Fr...
    Conventions:                     CF-1.11 ACDD-1.3
    standard_name_vocabulary:        CF Standard Name Table v81
    input_domain:                    Europe
    input_domain_id:                 EUR-12
    project:                         Socle-FR-Metro-Climat-2025
    ...                              ...
    time_coverage_duration:          P1Y
    time_coverage_resolution:        P1D
    product:                         bias-adjusted output
    product_version:                 v1
    comment:                         Preliminary data released in the Fall 20...
    keywords:                        humidity, climate, France, near-surface ...

In [15]:
with ProgressBar():
    ds = ds.compute()

[########################################] | 100% Completed | 108.83 s


In [16]:
ds

<xarray.Dataset> Size: 57MB
Dimensions:                      (location: 6, source_id: 17, time: 23376)
Coordinates:
  * location                     (location) object 48B 'paris' ... 'annecy'
  * source_id                    (source_id) object 136B 'EC-Earth3:r1i1p1f1:...
  * time                         (time) datetime64[ns] 187kB 1951-01-01T12:00...
    input_driving_source_id      (location, source_id) object 816B 'EC-Earth3...
    input_source_id              (location, source_id) object 816B 'RACMO23E'...
    lon                          (location) float64 48B 2.391 1.396 ... 6.1
    x                            (location) float64 48B 6.04e+05 ... 8.92e+05
    input_driving_variant_label  (location, source_id) object 816B 'r1i1p1f1'...
    lat                          (location) float64 48B 48.82 43.64 ... 45.88
    y                            (location) float64 48B 2.425e+06 ... 2.105e+06
Data variables:
    LambertParisII               (location, source_id) int32 408B 1 1 1 ... 1 1
    hussAdjust                   (location, source_id, time) float32 10MB 0.0...
    tasmaxAdjust                 (location, source_id, time) float32 10MB 270...
    sfcWindAdjust                (location, source_id, time) float32 10MB 4.2...
    prAdjust                     (location, source_id, time) float32 10MB 0.0...
    tasminAdjust                 (location, source_id, time) float32 10MB 266...
    tasAdjust                    (location, source_id, time) float32 10MB 268...
Attributes: (12/51)
    summary:                         Data were post-processed by the Météo-Fr...
    Conventions:                     CF-1.11 ACDD-1.3
    standard_name_vocabulary:        CF Standard Name Table v81
    input_domain:                    Europe
    input_domain_id:                 EUR-12
    project:                         Socle-FR-Metro-Climat-2025
    ...                              ...
    time_coverage_duration:          P1Y
    time_coverage_resolution:        P1D
    product:                         bias-adjusted output
    product_version:                 v1
    comment:                         Preliminary data released in the Fall 20...
    keywords:                        humidity, climate, France, near-surface ...

In [18]:
print(ds)

<xarray.Dataset> Size: 57MB
Dimensions:                      (location: 6, source_id: 17, time: 23376)
Coordinates:
  * location                     (location) object 48B 'paris' ... 'annecy'
  * source_id                    (source_id) object 136B 'EC-Earth3:r1i1p1f1:...
  * time                         (time) datetime64[ns] 187kB 1951-01-01T12:00...
    input_driving_source_id      (location, source_id) object 816B 'EC-Earth3...
    input_source_id              (location, source_id) object 816B 'RACMO23E'...
    lon                          (location) float64 48B 2.391 1.396 ... 6.1
    x                            (location) float64 48B 6.04e+05 ... 8.92e+05
    input_driving_variant_label  (location, source_id) object 816B 'r1i1p1f1'...
    lat                          (location) float64 48B 48.82 43.64 ... 45.88
    y                            (location) float64 48B 2.425e+06 ... 2.105e+06
Data variables:
    LambertParisII               (location, source_id) int32 408B 1 1 1 ...

In [19]:
# All string coords
string_coords = [
    "location",
    "source_id",
]

# All string data vars
string_vars = [
    "input_driving_source_id",
    "input_source_id",
    "input_driving_variant_label",
]

encoding = {v: {"dtype": "S1"} for v in string_coords + string_vars}

In [20]:
ds.to_netcdf("rcm-timeseries-per-location.nc", encoding=encoding)

In [ ]:
img = dt['NEX-GDDP-CMIP6/IPSL-CM6A-LR/historical/r1i1p1f1'].isel(time=0).tas.hvplot()
img = img.opts(width=1000, height=700, cmap='viridis')

x = pd.DataFrame(evaluation.locations).transpose()
x['location'] = x.index
print(len(x))
points = hv.Points(x).opts(size=10, marker='circle', color='red', tools=['hover'])

img * points